In [1]:
from random import random
import math
import numpy as np
import copy

In [2]:
def loadMovieLens(path='./data/movielens'):
    #Get movie titles
    movies={}
    rev_movies={}
    for idx,line in enumerate(open(path+'/u.item')):
        idx,title=line.split('|')[0:2]
        movies[idx]=title
        rev_movies[title]=idx

    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        
    return prefs,rev_movies

In [3]:
data,movies = loadMovieLens("data/ml-100k")

In [4]:
data['3']

{'187 (1997)': 2.0,
 'Air Force One (1997)': 2.0,
 'Alien: Resurrection (1997)': 3.0,
 'Apostle, The (1997)': 4.0,
 'Bean (1997)': 2.0,
 'Boogie Nights (1997)': 5.0,
 'Chasing Amy (1997)': 3.0,
 'Conspiracy Theory (1997)': 5.0,
 'Contact (1997)': 2.0,
 'Cop Land (1997)': 4.0,
 'Crash (1996)': 1.0,
 'Critical Care (1997)': 1.0,
 "Dante's Peak (1997)": 2.0,
 'Deconstructing Harry (1997)': 3.0,
 'Deep Rising (1998)': 1.0,
 'Desperate Measures (1998)': 4.0,
 "Devil's Advocate, The (1997)": 3.0,
 "Devil's Own, The (1997)": 1.0,
 'Edge, The (1997)': 4.0,
 'Event Horizon (1997)': 4.0,
 'Everyone Says I Love You (1996)': 2.0,
 'Fallen (1998)': 3.0,
 'G.I. Jane (1997)': 2.0,
 'Game, The (1997)': 2.0,
 'Good Will Hunting (1997)': 2.0,
 'Hard Rain (1998)': 3.0,
 'Hoodlum (1997)': 3.0,
 'House of Yes, The (1997)': 1.0,
 'How to Be a Player (1997)': 1.0,
 'In the Name of the Father (1993)': 2.0,
 'Jackie Brown (1997)': 5.0,
 'Kiss the Girls (1997)': 1.0,
 'L.A. Confidential (1997)': 2.0,
 'Liar Lia

In [5]:
def getRawArray(data):
    d = []
    for u in data.keys():
        for i in data[u].keys():
            d.append([u,i,data[u][i]])
    return np.array(d)

In [6]:
# splitting while avoiding to reduce the dataset too much
def split_train_test(data,percent_test):
    test={}
    train={}
    movie={}
    for u in data.keys():
        test.setdefault(u,{})
        train.setdefault(u,{})
        for movie in data[u]:
            #print(data[u][movie])
            if (random()<percent_test):
                test[u][movie]=data[u][movie]
            else:
                train[u][movie]=data[u][movie]
    return train, test

In [7]:
def split_train_test_by_movies(data,percent_test):
    test={}
    train={}
    movie={}
    for u in data.keys():
        for movie in data[u]:
            if (random()<percent_test):
                try:
                    test[movie][u]=data[u][movie]
                except KeyError:
                    test.setdefault(movie,{})
                    test[movie][u]=data[u][movie]
            else:
                try:
                    train[movie][u]=data[u][movie]
                except KeyError:
                    train.setdefault(movie,{})
                    train[movie][u]=data[u][movie]
    return train, test

In [8]:
percent_test=0.2
train,test=split_train_test(data,percent_test)

In [9]:
percent_test=0.2
m_train,m_test=split_train_test_by_movies(data,percent_test)

In [10]:
def deleteUnseenInTest(train,test):
    for k in test.keys():
        try:
            train[k]
        except KeyError:
            test.pop(k,None)

In [11]:
deleteUnseenInTest(train,test)
deleteUnseenInTest(m_train,m_test)

In [12]:
rawArray = getRawArray(data)
rawArrayTest = getRawArray(test)

---

## Baseline: mean by user

In [13]:
class baselineMeanMovie:
    def __init__(self):
        self.users={}
        self.movies={}
    def fit(self,train):
        movies = get_moove(train)
        for movie in movies:
            note=0.0
            cpt=0
            for user in train:
                try:
                    note+=train[user][movie]
                    cpt+=1
                except KeyError:
                    pass
            note=note/cpt
            self.movies[movie]=note
        
    def predict(self,user,movie):
        return self.movies[movie]
    def score(self,X):
        nb_movies = len(get_moove(X))
        score = 0.0
        for user in X:
            for movie in X[user]:
                score += (self.predict(user,movie) - X[user][movie])**2
        return score/nb_movies

In [14]:
class baselineMeanUser:
    def __init__(self):
        self.users={}
    def fit(self,train):
        for user in train.keys():
            note=0.0
            for movie in train[user].keys():
                note+=train[user][movie]
            note=note/len(train[user])
            self.users[user]=note
        
    def predict(self,users):
        return [self.users[u] for u in users]

In [15]:
baseline_mu= baselineMeanUser()
baseline_mu.fit(train)
pred = baseline_mu.predict(rawArray[:,0])
print("Mean Error %0.6f" %(
        (np.array(pred) - np.array(rawArray[:,2], float)) ** 2).mean())

Mean Error 1.065131


In [16]:
class baselineMeanMovie:
    def __init__(self):
        self.movies={}
    def fit(self,train):
        for movie in train.keys():
            note=0.0
            for user in train[movie].keys():
                note+=train[movie][user]
            note=note/len(train[movie])
            self.movies[movie]=note
        
    def predict(self,movies):
        res=[]
        for m in movies:
            try:
                res.append(self.movies[m])
            except:
                res.append(3)
        return res

In [17]:
baseline_mm= baselineMeanMovie()
baseline_mm.fit(m_train)
pred = baseline_mm.predict(rawArrayTest[:,1])
print("Mean Error %0.6f" %(
        (np.array(pred) - np.array(rawArrayTest[:,2], float)) ** 2).mean())

Mean Error 1.009774


In [18]:
m_test['Adventures of Pinocchio, The (1996)']

{'125': 4.0, '181': 1.0, '200': 3.0, '222': 2.0, '592': 2.0}

In [19]:
rawArray[:5]

array([['344', 'Birdcage, The (1996)', '4.0'],
       ['344', 'Enchanted April (1991)', '4.0'],
       ['344', 'Diabolique (1996)', '2.0'],
       ['344', 'Face/Off (1997)', '4.0'],
       ['344', 'My Fellow Americans (1996)', '3.0']], 
      dtype='|S81')

In [20]:
len(m_train['Birdcage, The (1996)'])

241

---

In [175]:
class matrixFactorisation():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=2000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.couples = couples
        self.loss = []
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                r = np.random.randint(len(couples))
                user = couples[r][0]
                item = couples[r][1]
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k)
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1)
                tmp = dataUsers[user][item] - self.p[user].dot(self.q[item])[0][0]
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                loss = loss + tmp*tmp #Sans régularisation
            self.loss.append(loss)
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    oprimP = False if optimQ else True
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

In [176]:
model3 = matrixFactorisation(10, alternate=0)
model3.fit(trainUsers, trainItems, trainCouples)

0 2.77841840868
100 1.26732246398
200 1.05346402861
300 0.983480018466
400 0.940916112566
500 0.912340589664
600 0.895851403893
700 0.880687032448


KeyboardInterrupt: 

In [22]:

dm = np.dok_matrix(train)

AttributeError: 'module' object has no attribute 'dok_matrix'

In [88]:
print(len(movies))
print(len(data.keys()))

1664
943


In [64]:
movies["Adventures of Pinocchio, The (1996)"]

'1060'

In [82]:
rawMatrix = np.zeros((len(data.keys())+1,1682+1))

In [83]:
np.shape(rawMatrix)

(944, 1683)

In [84]:
data["1"]["101 Dalmatians (1996)"]

2.0

In [85]:
for u in data:
    for m in data[u]:
        rawMatrix[int(u)][int(movies[m])] = data[u][m]

In [87]:
rawMatrix[:5][:5]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  3., ...,  0.,  0.,  0.],
       [ 0.,  4.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [98]:
import numpy as np
from scipy import linalg
from numpy import dot

def nmf(X, latent_features, max_iter=100, error_limit=1e-6, fit_error_limit=1e-6):
    """
    Decompose X to A*Y
    """
    eps = 1e-5
    print 'Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter)
    #X = X.toarray()  # I am passing in a scipy sparse matrix

    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # updates
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # evaluation
        if i % 50 == 0 or i == 1 or i == max_iter:
            print 'Iteration {}:'.format(i),
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

            curRes = linalg.norm(mask * (X - X_est), ord='fro')
            print 'fit residual', np.round(fit_residual, 4),
            print 'total residual', np.round(curRes, 4)
            if curRes < error_limit or fit_residual < fit_error_limit:
                break

    return A, Y

In [104]:
A,Y = nmf(rawMatrix,10,max_iter=1000)

Starting NMF decomposition with 10 latent features and 1000 iterations.
Iteration 1: fit residual 882.6755 total residual 336.9268
Iteration 50: fit residual 202.99 total residual 255.134
Iteration 100: fit residual 54.7172 total residual 245.6577
Iteration 150: fit residual 33.4924 total residual 241.3665
Iteration 200: fit residual 25.0348 total residual 238.6455
Iteration 250: fit residual 21.4902 total residual 236.5486
Iteration 300: fit residual 17.9908 total residual 234.943
Iteration 350: fit residual 15.3883 total residual 233.6517
Iteration 400: fit residual 13.878 total residual 232.545
Iteration 450: fit residual 12.6491 total residual 231.5866
Iteration 500: fit residual 11.4608 total residual 230.7769
Iteration 550: fit residual 10.3756 total residual 230.0917
Iteration 600: fit residual 9.5832 total residual 229.4888
Iteration 650: fit residual 9.25 total residual 228.9448
Iteration 700: fit residual 8.5896 total residual 228.4733
Iteration 750: fit residual 8.0066 total

In [105]:
resMatrix = A.dot(Y)

In [150]:
class evalMF:
    def __init__(self,resMatrix,dicU,dicI):
        self.resMatrix=resMatrix
        self.dicU = dicU
        self.dicI = dicI
    def fit(self):
        pass
        
    def predict(self,user,movie):
        #res=[]
        #for m in movies:
        #    try:
        #        res.append(self.movies[m])
        #    except:
        #        res.append(3)
        #return res
        return self.resMatrix[int(user)][int(self.dicI[movie])]

In [152]:
mf= evalMF(resMatrix,data,movies)
#baseline_mm.fit(m_train)
#pred = baseline_mm.predict(rawArrayTest[:,1])
#print("Mean Error %0.6f" %(
#        (np.array(pred) - np.array(rawArrayTest[:,2], float)) ** 2).mean())

In [153]:
mf.predict("1","Akira (1988)")

4.2954887468335441

In [159]:
mf.predict("1","All Dogs Go to Heaven 2 (1996)")

1.3539530296470097

In [ ]:
print("Mean Error %0.6f" %(
        (np.array(pred) - np.array(rawArrayTest[:,2], float)) ** 2).mean())

In [160]:
rawArrayTest[:5]

array([['344', 'Birdcage, The (1996)', '4.0'],
       ['344', 'Emma (1996)', '4.0'],
       ['344', 'Diabolique (1996)', '2.0'],
       ['344', 'Sleepers (1996)', '4.0'],
       ['344', 'Breakdown (1997)', '3.0']], 
      dtype='|S79')

In [167]:
np.array([ (float(ra[2]) - mf.predict(ra[0],ra[1]))**2 for ra in rawArrayTest]).mean()

0.51039178926982265

In [154]:
train["1"]

{'101 Dalmatians (1996)': 2.0,
 '20,000 Leagues Under the Sea (1954)': 3.0,
 '2001: A Space Odyssey (1968)': 4.0,
 'Abyss, The (1989)': 3.0,
 'Ace Ventura: Pet Detective (1994)': 3.0,
 'Air Bud (1997)': 1.0,
 'Aladdin (1992)': 4.0,
 'Alien (1979)': 5.0,
 'Aliens (1986)': 5.0,
 'All Dogs Go to Heaven 2 (1996)': 1.0,
 'Angels and Insects (1995)': 4.0,
 "Antonia's Line (1995)": 5.0,
 'Apocalypse Now (1979)': 3.0,
 'Apollo 13 (1995)': 4.0,
 'Aristocats, The (1970)': 2.0,
 'Army of Darkness (1993)': 4.0,
 'Austin Powers: International Man of Mystery (1997)': 4.0,
 'Babe (1995)': 1.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 2.0,
 'Basic Instinct (1992)': 3.0,
 'Batman & Robin (1997)': 1.0,
 'Batman Returns (1992)': 1.0,
 'Beavis and Butt-head Do America (1996)': 3.0,
 'Belle de jour (1967)': 3.0,
 'Big Night (1996)': 5.0,
 'Billy Madison (1995)': 2.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 5.0,
 'Blues Brothers, The (1980)': 4.0,
 'Bound (1996)': 5.0,
 "Bram Sto

In [121]:
test["1"]

{'12 Angry Men (1957)': 5.0,
 'Akira (1988)': 4.0,
 'Amadeus (1984)': 5.0,
 'Batman Forever (1995)': 1.0,
 'Bedknobs and Broomsticks (1971)': 2.0,
 'Breaking the Waves (1996)': 5.0,
 'Brothers McMullen, The (1995)': 3.0,
 'Cable Guy, The (1996)': 3.0,
 'Chasing Amy (1997)': 5.0,
 'Cyrano de Bergerac (1990)': 5.0,
 'Dead Poets Society (1989)': 5.0,
 'Dirty Dancing (1987)': 2.0,
 'Disclosure (1994)': 4.0,
 'Ed Wood (1994)': 4.0,
 'Exotica (1994)': 4.0,
 'Fish Called Wanda, A (1988)': 3.0,
 'Forrest Gump (1994)': 3.0,
 'Frighteners, The (1996)': 4.0,
 'Full Metal Jacket (1987)': 3.0,
 'Full Monty, The (1997)': 5.0,
 'Gone with the Wind (1939)': 4.0,
 'Grand Day Out, A (1992)': 3.0,
 'Hudsucker Proxy, The (1994)': 5.0,
 'I.Q. (1994)': 3.0,
 'M*A*S*H (1970)': 3.0,
 'Mad Love (1995)': 2.0,
 'Mask, The (1994)': 4.0,
 'Maverick (1994)': 3.0,
 'Maya Lin: A Strong Clear Vision (1994)': 5.0,
 'Mimic (1997)': 2.0,
 'Nightmare Before Christmas, The (1993)': 5.0,
 'Nikita (La Femme Nikita) (1990)': 

---

In [123]:
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

In [124]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]

R = numpy.array(R)

N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = numpy.dot(nP, nQ.T)

In [125]:
nR

array([[ 4.99650255,  2.93418578,  3.99234219,  0.99820419],
       [ 3.96298764,  2.33573468,  3.36035393,  0.99707133],
       [ 1.0716961 ,  0.82538354,  5.337668  ,  4.96193011],
       [ 0.9633362 ,  0.72181627,  4.33820223,  3.97311633],
       [ 1.81106643,  1.21517699,  4.91344271,  4.03428495]])

---

In [140]:
%%time

R = rawMatrix

N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K, steps=100)
nR = numpy.dot(nP, nQ.T)

CPU times: user 12min 15s, sys: 5.83 s, total: 12min 21s
Wall time: 12min 33s


In [141]:
nR[:5,:5]

array([[ 0.28872932,  2.18817177,  1.85474169,  1.70232665,  1.99304327],
       [ 0.52041677,  4.00343944,  3.38358237,  3.14717666,  3.64726599],
       [ 0.5200989 ,  3.97133365,  3.36127841,  3.10588446,  3.61760816],
       [ 0.40158254,  3.22346401,  2.70251611,  2.60665046,  2.93853012],
       [ 0.60854536,  4.61584397,  3.9118434 ,  3.59312263,  4.2042843 ]])

In [142]:
rawMatrix[:5,:5]

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  5.,  3.,  4.,  3.],
       [ 0.,  4.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [168]:
mf= evalMF(nR,data,movies)
mf.predict("1","Akira (1988)")

3.6673599590921162

In [169]:
np.array([ (float(ra[2]) - mf.predict(ra[0],ra[1]))**2 for ra in rawArrayTest]).mean()

0.86354390536474346